In [ ]:
%load_ext autoreload
%autoreload 2
from src.functions import create_model, plot_data, check_gpus, create_train_val_datasets, load_stations_from_path, create_test_datasets

In [ ]:
check_gpus()

In [ ]:
TRAIN_PATH = '../data/labeled_daily/train'
TEST_PATH = '../data/labeled_daily/test'
SEQUENCE_LENGTH = 20
TARGET_START_INDEX = SEQUENCE_LENGTH - 1
FEATURE_COLUMNS = ['HS', 'day_sin', 'day_cos', 'month_sin', 'month_cos']
TARGET_COLUMN = 'no_snow'
DATE_COLUMN = 'measure_date'
SPLIT_PERCENTAGE = 0.8
DATASET_BATCH_SIZE = 64
# DATASET_BATCH_SIZE = None

# Active learning
UNCERTAINTY_ITERATIONS = 5

# Model configuration
MODEL_ARCHITECTURE = "128(l)-64-8(d)-1"
MODEL_INPUT_SHAPE = (SEQUENCE_LENGTH, len(FEATURE_COLUMNS))
MODEL_DROPOUT_RATE = 0.5
MODEL_OPTIMIZER = 'adam'
MODEL_METRICS = ['accuracy']
MODEL_LOSS = 'binary_crossentropy'
MODEL_BATCH_SIZE = 64
MODEL_EPOCHS = 10

In [ ]:
training_stations = load_stations_from_path(TRAIN_PATH)
testing_stations = load_stations_from_path(TEST_PATH)

In [ ]:
train_dataset, val_dataset, mean, std = create_train_val_datasets(
    training_stations, SPLIT_PERCENTAGE, FEATURE_COLUMNS, TARGET_COLUMN, SEQUENCE_LENGTH, TARGET_START_INDEX, DATASET_BATCH_SIZE
)

In [ ]:
test_datasets = create_test_datasets(
    testing_stations, FEATURE_COLUMNS, TARGET_COLUMN, SEQUENCE_LENGTH, TARGET_START_INDEX, DATASET_BATCH_SIZE, mean, std
)

In [ ]:
model = create_model(MODEL_ARCHITECTURE, MODEL_INPUT_SHAPE, MODEL_DROPOUT_RATE)
model.compile(
    optimizer=MODEL_OPTIMIZER,
    metrics=MODEL_METRICS,
    loss=MODEL_LOSS
)

history = model.fit(
    train_dataset,
    epochs=MODEL_EPOCHS,
    batch_size=MODEL_BATCH_SIZE,
    validation_data=val_dataset,
)

In [ ]:
for dt in test_datasets:
    model.evaluate(dt)

In [ ]:
predictions = [model.predict(td).reshape((-1,)) > 0.5 for td in test_datasets]

plot_data(
    [test_station[TARGET_START_INDEX:] for test_station in testing_stations],
    predictions=predictions
)